In [3]:
import torch
a=torch.randn(3,4)
a[torch.arange(2), :torch.ra]

tensor([[-0.4928, -0.6266,  1.6426,  1.0207],
        [-0.6854, -0.9873,  1.0441, -0.8110],
        [-0.3582,  0.2052, -0.0492, -0.9293]])

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='0'

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('waveglow/')

import matplotlib.pyplot as plt
%matplotlib inline

import IPython.display as ipd
import pickle as pkl
import torch
import hparams
from torch.utils.data import DataLoader
from modules.model import Model
from text import text_to_sequence, sequence_to_text
from denoiser import Denoiser
from tqdm import tqdm_notebook as tqdm
import librosa
from modules.loss import MDNLoss

data_type = 'phone'
checkpoint_path = f"training_log/aligntts/checkpoint_40000"
state_dict = {}
for k, v in torch.load(checkpoint_path)['state_dict'].items():
    state_dict[k[7:]]=v


model = Model(hparams).cuda()
model.load_state_dict(state_dict)
_ = model.cuda().eval()
criterion = MDNLoss()

In [2]:
datasets = ['train', 'val', 'test']

for dataset in datasets:
    with open(f'filelists/ljs_audio_text_{dataset}_filelist.txt', 'r') as f:
        lines = [line.split('|') for line in f.read().splitlines()]

    for i in tqdm(range(len(lines))):
        file_name, _, text = lines[i]
        text = '^' + text + '~'
        seq = os.path.join('../Dataset/LJSpeech-1.1/preprocessed',
                           f'{data_type}_seq')
        mel = os.path.join('../Dataset/LJSpeech-1.1/preprocessed',
                           'melspectrogram')

        with open(f'{seq}/{file_name}_sequence.pkl', 'rb') as f:
            text_padded = pkl.load(f).unsqueeze(0).cuda()
        with open(f'{mel}/{file_name}_melspectrogram.pkl', 'rb') as f:
            mel_padded = pkl.load(f).unsqueeze(0).cuda()
        
        mel_padded = (mel_padded - torch.min(mel_padded))\
                         / torch.max((mel_padded - torch.min(mel_padded)))
        
        text_lengths=torch.LongTensor([text_padded.size(1)]).cuda()
        mel_lengths=torch.LongTensor([mel_padded.size(2)]).cuda()
        
        

        encoder_input = model.Prenet(text_padded)
        hidden_states, _ = model.FFT_lower(encoder_input, text_lengths)
        mu_sigma = model.get_mu_sigma(hidden_states)
        _, log_prob_matrix = criterion(mu_sigma, mel_padded, text_lengths, mel_lengths)
    
        alignments = model.viterbi(log_prob_matrix[0:1], text_lengths[0:1], mel_lengths[0:1])[0].t()
        
        with open(f'../Dataset/LJSpeech-1.1/preprocessed/alignments/{file_name}.pkl', 'wb') as f:
            pkl.dump(alignments, f)